#### Implementation of Doolittle LU decomposition to find L, U

In [1]:
import numpy as np

In [2]:
def doolittle(A):

    n = A.shape[0]
    L = np.zeros((n,n), np.float64)
    U = np.zeros((n,n), np.float64)

    for i in range(n):
        # U
        for k in range(i,n):
            s1 = 0  # summation of L(i, j)*U(j, k) 
            for j in range(i):
                s1 += L[i,j]*U[j,k]
            U[i,k] = A[i,k] - s1

        # L
        for k in range(i,n):
            if i==k:
                # diagonal terms of L 
                L[i,i] = 1
            else:
                s2 = 0 # summation of L(k, j)*U(j, i) 
                for j in range(i):
                    s2 += L[k,j]*U[j,i]
                L[k,i] = (A[k,i] - s2)/U[i,i]

    return L, U

#### Solving equations after LU factorization
_Implementation of forward substitution_

In [3]:
def forward_substitution(L, b):
    
    # Get number of rows
    n = L.shape[0]
    
    # Allocating space for the solution vector
    y = np.zeros_like(b, dtype=np.double);
    
    # Here we perform the forward-substitution.  
    # Initializing  with the first row.
    y[0] = b[0] / L[0, 0]
    
    # Looping over rows in reverse (from the bottom  up), 
    # starting with the second to last row, because  the 
    # last row solve was completed in the last step.
    for i in range(1, n):
        y[i] = (b[i] - np.dot(L[i,:i], y[:i])) / L[i,i]
        
    return y

_Implementation of back substitution_

In [4]:
def back_substitution(U, y):
    
    # Number of rows
    n = U.shape[0]
    
    # Allocating space for the solution vector
    x = np.zeros_like(y, dtype=np.double);

    # Here we perform the back-substitution.  
    # Initializing with the last row.
    x[-1] = y[-1] / U[-1, -1]
    
    # Looping over rows in reverse (from the bottom up), 
    # starting with the second to last row, because the 
    # last row solve was completed in the last step.
    for i in range(n-2, -1, -1):
        x[i] = (y[i] - np.dot(U[i,i:], x[i:])) / U[i,i]
        
    return x

#### Overall equation solution

In [5]:
import pprint as pp
def lu_solve(A, b):
    
    L, U = doolittle(A)
    
    print("Lower diagonal matrix: \n")
    pp.pprint(L)
    print('\n')
    print("Upper diagonal matrix: \n")
    pp.pprint(U)
    print('\n')
    
    y = forward_substitution(L, b)
    
    x = back_substitution(U,y)
    print("Solution: \n")
    pp.pprint(x)
    
    return

In [6]:
A = np.array([[3.,-1.,4.],[-2.,0.,5.],[7.,2.,-2.]])
b = np.array([[6.,-4.],[3.,2.],[7.,-5.]])

In [7]:
lu_solve(A,b)

Lower diagonal matrix: 

array([[ 1.        ,  0.        ,  0.        ],
       [-0.66666667,  1.        ,  0.        ],
       [ 2.33333333, -6.5       ,  1.        ]])


Upper diagonal matrix: 

array([[ 3.        , -1.        ,  4.        ],
       [ 0.        , -0.66666667,  7.66666667],
       [ 0.        ,  0.        , 38.5       ]])


Solution: 

array([[ 1.00000000e+00, -1.00000000e+00],
       [ 1.00000000e+00,  1.00000000e+00],
       [ 1.00000000e+00,  2.30695693e-17]])


<br><br>
#### Inverse of A 

In [8]:
n = A.shape[0]
I = np.identity(n)

Modifying lu_solve function

In [9]:
def lu_inverse(A, I):
    
    L, U = doolittle(A)
    
    y = forward_substitution(L, I)
    
    x = back_substitution(U,y)
#     print("Inverse of A: \n")
#     pp.pprint(x)
    
    return x

In [10]:
lu_inverse(A,I)

array([[ 0.12987013, -0.07792208,  0.06493506],
       [-0.4025974 ,  0.44155844,  0.2987013 ],
       [ 0.05194805,  0.16883117,  0.02597403]])

In [11]:
a = lu_inverse(A,I)
np.dot(A,a).round()

array([[ 1.,  0., -0.],
       [-0.,  1., -0.],
       [ 0., -0.,  1.]])